In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import os, re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from gensim.models.doc2vec import TaggedDocument
import pickle
from gensim.parsing.preprocessing import preprocess_string

from gensim.parsing.preprocessing import strip_tags
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_numeric

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hesham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
data = pd.read_csv('./data_clean.csv')

In [10]:
data.head()

,url,label,http,text
0,retroplanet.com,Antiques and Collectibles/1950s Memorabilia,http://retroplanet.com,"Retro Planet ~ Tin Signs, Retro Decor, Diner F..."
1,hepcat.se,Antiques and Collectibles/1950s Memorabilia,http://hepcat.se,"HepCat Store - Clothing, shoes, hats and acces..."
2,barkclothhawaii.com,Antiques and Collectibles/1950s Memorabilia,http://barkclothhawaii.com,"Barkcloth Hawaii ""Vintage"" Tropical Hawaiian F..."
3,classic-modern.co.uk,Antiques and Collectibles/1950s Memorabilia,http://classic-modern.co.uk,Classic Modern Midcentury Design We use cookie...
4,hiandlomodern.com,Antiques and Collectibles/1950s Memorabilia,http://hiandlomodern.com,Hi+LoModern - 20th Century Artifacts hi+lomode...


In [11]:
data.describe(include='all')[:4]

,url,label,http,text
count,38003,38003,38003,38003
unique,38003,3529,38003,38003
top,miamiaromatherapy.com,Gifts/Personalized/Trophies,http://japangarden.co.uk,Rock Guitar Lessons Online Site Home About Con...
freq,1,50,1,1


In [12]:
stop_words = set(stopwords.words('english'))

p_stemmer = PorterStemmer()

def preprocess(text):    
    text = re.sub(r'[^\x20-\x7E]+',' ', text).lower()
    text = strip_tags(text)
    text = " ".join(w for w in text.split() if 'http' not in w and 'www' not in w and '@' not in w and not re.match('.+\.[a-z]{2,3}', w))
    text = strip_punctuation(text)
    text = strip_multiple_whitespaces(text)
    text = strip_numeric(text)
    text = strip_short(text)
    tokens = [i for i in text.split() if i not in stop_words]
    tokens_alnum = [i for i in tokens if i.isalnum()]
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in tokens_alnum]
    # remove empty
    filtered = [i for i in stemmed_tokens if len(i) > 1]

    return filtered

In [13]:
docs = []

word_thresh = 15

english_thresh = 0.2

for idx, row in data.iterrows():
    processed = preprocess(row.text)
    if len(set(processed)) > word_thresh:
        wc = len(row.text.split())
        r = len(processed)/wc
        # Frensh and German websites will still match with each other
        if r > english_thresh:
            docs.append(TaggedDocument(processed, [row.url]))
print(len(docs), ' total documents.')

37092  total documents.


In [14]:
with open('./docs.pkl', 'wb') as file:
    pickle.dump(docs, file)

In [ ]:
# tag = ''
# for doc in docs:
#     if tag in doc.tags:
#         print(doc.words)